# Команда 7, кейс 7 — предсказание отказов оборудования

В этом ноутбуке:
1. Проводим EDA и проверяем качество данных.
2. Обосновываем и реализуем feature engineering.
3. Строим baseline-модель (Logistic Regression).
4. Сравниваем несколько моделей (дерево, SVM, RF, GBM, XGBoost, LightGBM)
с учётом сильного дисбаланса классов.
5. Проверяем влияние удаления исходных признаков Torque и Rotational_speed.
6. Дополнительно решаем задачу мультиклассовой классификации типа отказа.


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    cross_validate,
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    RocCurveDisplay,
    PrecisionRecallDisplay,
)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

sns.set(style="whitegrid")


## 1. Загрузка данных

В соответствии с замечанием эксперта **не используем test.csv**,
так как для него отсутствует целевая переменная и проверить качество модели
по нему невозможно. Вся работа проводится только на `train.csv`.


In [ ]:
data = pd.read_csv("train.csv")
data.head()


In [ ]:
data.info()


In [ ]:
# Проверим базовое описание числовых признаков
data.describe().T


## 2. Целевая переменная и дисбаланс классов

Целевая переменная — `Machine failure` (0 — всё нормально, 1 — произошёл отказ).
Набор данных явно несбалансирован, поэтому сразу будем это учитывать
при выборе метрик и моделей.


In [ ]:
target_col = "Machine failure"
failure_rate = data[target_col].value_counts(normalize=True).sort_index()
print("Распределение классов:")
print(failure_rate)

plt.figure(figsize=(4, 4))
failure_rate.plot(kind="bar")
plt.title("Доля отказов / неотказов")
plt.xticks([0, 1], ["Работает", "Отказ"], rotation=0)
plt.ylabel("Доля объектов")
plt.show()


Видим, что отказы составляют всего ~1.5–1.6% объектов — сильный дисбаланс.
Поэтому основной упор делаем на метрики:
- **ROC-AUC**
- **PR-AUC (average precision)**
- **Recall / Precision / F1** по положительному классу.



## 3. EDA: распределения признаков с учётом целевой

Важное замечание эксперта — гистограммы нужно **нормировать**, иначе они
плохо читаются. Используем `stat="density"` и `common_norm=False`,
чтобы по оси Y была не абсолютная частота, а *плотность* (доля объектов).


In [ ]:
numeric_cols = [
    "Air temperature [K]",
    "Process temperature [K]",
    "Rotational speed [rpm]",
    "Torque [Nm]",
    "Tool wear [min]",
]

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for ax, col in zip(axes, numeric_cols):
    sns.histplot(
        data=data,
        x=col,
        hue=target_col,
        stat="density",         # <-- нормировка
        common_norm=False,      # отдельная нормировка для каждого класса
        kde=False,
        ax=ax,
        bins=50,
    )
    ax.set_title(col)

fig.suptitle("Нормированные распределения признаков по классам Machine failure", fontsize=14)
plt.tight_layout()
plt.show()


По графикам видно:
- У отказов чуть смещены распределения по скорости и моменту.
- Износ инструмента у отказов, как правило, больше.
- Разница температур и абсолютные температуры тоже ведут себя по-разному.

Это подсказывает идеи для **feature engineering**.



## 4. Корреляции и логика признаков

Перед инжинирингом посмотрим на корреляции между основными признаками
и целевой переменной. Для этого уберём идентификатор и категориальные
столбцы, а также флаги типов отказов (они будут разобраны отдельно).


In [ ]:
flag_cols = ["TWF", "HDF", "PWF", "OSF", "RNF"]

corr_cols = [
    "Air temperature [K]",
    "Process temperature [K]",
    "Rotational speed [rpm]",
    "Torque [Nm]",
    "Tool wear [min]",
    target_col,
] + flag_cols

corr = data[corr_cols].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=False, cmap="coolwarm", center=0)
plt.title("Корреляционная матрица исходных признаков")
plt.show()


Выводы:
- `Torque [Nm]` и `Rotational speed [rpm]` заметно коррелируют с отказами
и между собой — это логично, так как их произведение даёт *мощность*.
- `Tool wear [min]` умеренно связан с отказами — накопленный износ.
- Флаги типов отказов (`TWF`, `HDF`, `PWF`, `OSF`, `RNF`) сильно коррелируют
с `Machine failure` — они описывают **причину уже произошедшего отказа**,
а не входные параметры, доступные заранее. Использовать их как признаки
для предсказания будущего отказа нельзя: это **утечка таргета**.

Далее эти флаги **не будем использовать как признаки** в основной модели.



## 5. Feature Engineering — обоснование

Инженерные признаки строим исходя из физического смысла:

1. **Разность температур**:
```python
Temperature_diff = Process temperature [K] - Air temperature [K]
```
Это мера перегрева оборудования относительно окружающей среды.
Высокая разность может указывать на проблемы с охлаждением.

2. **Мощность**:
```python
Power = Rotational speed [rpm] * Torque [Nm]
```
Произведение момента на угловую скорость — физически это мощность.
Высокая мощность = высокая нагрузка на систему.

3. **Взаимодействие нагрузки и износа**:
```python
Torque_wear = Torque [Nm] * Tool wear [min]
```
Даже умеренная нагрузка может быть опасна при большом износе инструмента.
Этот признак моделирует «нагрузку, накопленную во времени».

При этом **не торопимся удалять исходные признаки** `Torque` и
`Rotational speed` — как справедливо отметил эксперт, древовидные модели
(RandomForest, GradientBoosting, LightGBM) могут использовать в разбиениях
исходные переменные иначе, чем их произведение. Мы отдельно сравним
качество моделей с исходными признаками и без них.



In [ ]:
data_fe = data.copy()

# Переименуем столбцы для удобства (без пробелов и скобок)
data_fe = data_fe.rename(
    columns={
        "Product ID": "product_id",
        "Type": "type",
        "Air temperature [K]": "air_temp_k",
        "Process temperature [K]": "proc_temp_k",
        "Rotational speed [rpm]": "rot_speed_rpm",
        "Torque [Nm]": "torque_nm",
        "Tool wear [min]": "tool_wear_min",
        "Machine failure": "machine_failure",
    }
)

# 1. Разность температур
data_fe["temp_diff"] = data_fe["proc_temp_k"] - data_fe["air_temp_k"]

# 2. Мощность
data_fe["power"] = data_fe["rot_speed_rpm"] * data_fe["torque_nm"]

# 3. Взаимодействие нагрузки и износа
data_fe["torque_wear"] = data_fe["torque_nm"] * data_fe["tool_wear_min"]

data_fe.head()


Проверим, как новые признаки коррелируют с целевой.


In [ ]:
corr_fe_cols = [
    "air_temp_k",
    "proc_temp_k",
    "rot_speed_rpm",
    "torque_nm",
    "tool_wear_min",
    "temp_diff",
    "power",
    "torque_wear",
    "machine_failure",
]

plt.figure(figsize=(10, 8))
sns.heatmap(data_fe[corr_fe_cols].corr(), annot=False, cmap="coolwarm", center=0)
plt.title("Корреляции с инженерными признаками")
plt.show()


Видно, что:
- `power` и `torque_wear` заметно связаны с `machine_failure`;
- часть информации дублируется, но это нормально — модели сами выберут
полезные комбинации. Для эксперимента будем смотреть два варианта:
1. **Все признаки** (исходные + инженерные).
2. **Без исходных `torque_nm` и `rot_speed_rpm`** (только их комбинации).



## 6. Подготовка данных для модели (бинарная классификация)

Важные решения:
- **Не используем** флаги `TWF/HDF/PWF/OSF/RNF` как входные признаки
(они известны только после отказа).
- Убираем `id` и `product_id` как идентификаторы.
- Категориальный признак `type` кодируем **One-Hot**.
- Числовые признаки стандартизируем (важно для логистической регрессии и SVM).
- Используем стратифицированные разбиения и фиксируем `random_state`
для воспроизводимости.


In [ ]:
# Список флагов типа отказа
flag_cols = ["TWF", "HDF", "PWF", "OSF", "RNF"]

# Базовый набор признаков (с инженерными фичами)
feature_cols_base = [
    "air_temp_k",
    "proc_temp_k",
    "rot_speed_rpm",
    "torque_nm",
    "tool_wear_min",
    "temp_diff",
    "power",
    "torque_wear",
    "type",
]

X_full = data_fe[feature_cols_base].copy()
y = data_fe["machine_failure"]

# Вариант без исходных torque & rot_speed
feature_cols_no_torque_speed = [
    "air_temp_k",
    "proc_temp_k",
    "tool_wear_min",
    "temp_diff",
    "power",
    "torque_wear",
    "type",
]
X_no_ts = data_fe[feature_cols_no_torque_speed].copy()

# train/valid split (стратифицированный)
X_train_full, X_valid_full, y_train, y_valid = train_test_split(
    X_full,
    y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE,
)

X_train_no_ts, X_valid_no_ts, _, _ = train_test_split(
    X_no_ts,
    y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE,
)

# Преобразователи признаков
def make_preprocessor(X):
    numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
    cat_features = X.select_dtypes(include=["object"]).columns.tolist()

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_features),
            ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), cat_features),
        ]
    )
    return preprocessor, numeric_features, cat_features

preprocess_full, num_full, cat_full = make_preprocessor(X_train_full)
preprocess_no_ts, num_no_ts, cat_no_ts = make_preprocessor(X_train_no_ts)

print("Числовые признаки:", num_full)
print("Категориальные признаки:", cat_full)



## 7. Baseline: Логистическая регрессия

По рекомендации эксперта начинаем именно с **логистической регрессии**.
Используем:
- `class_weight="balanced"` — чтобы модель уделяла больше внимания редкому классу;
- стратифицированную кросс-валидацию;
- набор метрик: ROC-AUC, PR-AUC, Recall, Precision, F1.


In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scoring = ["roc_auc", "average_precision", "precision", "recall", "f1"]

def evaluate_model_cv(name, model, X, y, preprocessor, use_smote=False):
    """
    Обёртка: кросс-валидация с одинаковыми метриками для разных моделей.
    При необходимости добавляем SMOTE внутрь пайплайна.
    """
    if use_smote:
        pipe = ImbPipeline(
            steps=[
                ("preprocess", preprocessor),
                ("smote", SMOTE(random_state=RANDOM_STATE)),
                ("model", model),
            ]
        )
    else:
        pipe = Pipeline(
            steps=[
                ("preprocess", preprocessor),
                ("model", model),
            ]
        )

    cv_results = cross_validate(
        pipe,
        X,
        y,
        cv=cv,
        scoring=scoring,
        n_jobs=-1,
    )
    summary = {metric: cv_results[f"test_{metric}"].mean() for metric in scoring}
    print(f"
{name}")
    for m, v in summary.items():
        print(f"{m:>18}: {v:.4f}")
    return summary

baseline_logreg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
)

logreg_scores = evaluate_model_cv(
    "Logistic Regression (baseline, все признаки)",
    baseline_logreg,
    X_full,
    y,
    preprocess_full,
    use_smote=False,
)


Для логистической регрессии также посмотрим на качество на отложенной выборке
и на разбиение по порогу 0.5.


In [ ]:
# Обучаем пайплайн на train_full и проверяем на valid_full
logreg_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess_full),
        ("model", baseline_logreg),
    ]
)
logreg_pipeline.fit(X_train_full, y_train)

y_proba_valid = logreg_pipeline.predict_proba(X_valid_full)[:, 1]
y_pred_valid = (y_proba_valid >= 0.5).astype(int)

print("Classification report (valid):")
print(classification_report(y_valid, y_pred_valid, digits=4))

cm = confusion_matrix(y_valid, y_pred_valid)
plt.figure(figsize=(4, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion matrix — Logistic Regression (valid)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

RocCurveDisplay.from_predictions(y_valid, y_proba_valid)
plt.title("ROC-кривая — Logistic Regression")
plt.show()

PrecisionRecallDisplay.from_predictions(y_valid, y_proba_valid)
plt.title("PR-кривая — Logistic Regression")
plt.show()



## 8. Сравнение нескольких моделей

Реализуем единый цикл по моделям:
- Decision Tree
- SVM
- Random Forest
- Gradient Boosting
- XGBoost
- LightGBM

Для несбалансированных данных используем либо `class_weight`, либо
параметры `scale_pos_weight` / `is_unbalance`.


In [ ]:
models = []

# 1. Decision Tree
models.append(
    (
        "DecisionTree",
        DecisionTreeClassifier(
            max_depth=6,
            min_samples_leaf=50,
            class_weight="balanced",
            random_state=RANDOM_STATE,
        ),
        False,  # SMOTE не нужен, есть class_weight
    )
)

# 2. SVM (RBF)
models.append(
    (
        "SVC (RBF)",
        SVC(
            kernel="rbf",
            C=1.0,
            gamma="scale",
            probability=True,
            class_weight="balanced",
            random_state=RANDOM_STATE,
        ),
        False,
    )
)

# 3. Random Forest
models.append(
    (
        "RandomForest",
        RandomForestClassifier(
            n_estimators=300,
            max_depth=None,
            min_samples_leaf=20,
            n_jobs=-1,
            class_weight="balanced_subsample",
            random_state=RANDOM_STATE,
        ),
        False,
    )
)

# 4. Gradient Boosting (sklearn)
models.append(
    (
        "GradientBoosting",
        GradientBoostingClassifier(
            learning_rate=0.05,
            n_estimators=200,
            max_depth=3,
            random_state=RANDOM_STATE,
        ),
        False,
    )
)

# 5. XGBoost
pos_weight = (y == 0).sum() / (y == 1).sum()
models.append(
    (
        "XGBClassifier",
        XGBClassifier(
            n_estimators=300,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.8,
            colsample_bytree=0.8,
            scale_pos_weight=pos_weight,
            objective="binary:logistic",
            eval_metric="logloss",
            n_jobs=-1,
            random_state=RANDOM_STATE,
        ),
        False,
    )
)

# 6. LightGBM
models.append(
    (
        "LGBMClassifier",
        LGBMClassifier(
            n_estimators=400,
            learning_rate=0.05,
            max_depth=-1,
            num_leaves=31,
            subsample=0.8,
            colsample_bytree=0.8,
            class_weight="balanced",
            random_state=RANDOM_STATE,
        ),
        False,
    )
)

results = []
for name, model, use_smote in models:
    scores = evaluate_model_cv(
        name,
        model,
        X_full,
        y,
        preprocess_full,
        use_smote=use_smote,
    )
    scores["model"] = name
    results.append(scores)

results_df = pd.DataFrame(results).set_index("model")
results_df.sort_values("roc_auc", ascending=False)


По таблице можно выбрать лучшую модель (обычно LightGBM / XGBoost /
RandomForest показывают лучший компромисс между ROC-AUC и PR-AUC).
Для дальнейшего анализа выберем, например, **LightGBM** как финальную модель.



In [ ]:
best_model = LGBMClassifier(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight="balanced",
    random_state=RANDOM_STATE,
)

best_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess_full),
        ("model", best_model),
    ]
)

best_pipeline.fit(X_train_full, y_train)

y_proba_best = best_pipeline.predict_proba(X_valid_full)[:, 1]
y_pred_best = (y_proba_best >= 0.5).astype(int)

print("Финальная модель: LightGBM")
print(classification_report(y_valid, y_pred_best, digits=4))

cm_best = confusion_matrix(y_valid, y_pred_best)
plt.figure(figsize=(4, 4))
sns.heatmap(cm_best, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion matrix — LightGBM (valid)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

RocCurveDisplay.from_predictions(y_valid, y_proba_best)
plt.title("ROC-кривая — LightGBM")
plt.show()

PrecisionRecallDisplay.from_predictions(y_valid, y_proba_best)
plt.title("PR-кривая — LightGBM")
plt.show()



## 9. Влияние удаления Torque и Rotational speed

Реализуем тот же набор моделей (для краткости — логистическая регрессия
и RandomForest) на наборе признаков **без** `torque_nm` и `rot_speed_rpm`.
Это ответ на комментарий эксперта «не торопиться удалять исходные признаки».


In [ ]:
logreg_no_ts = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
)
logreg_no_ts_scores = evaluate_model_cv(
    "Logistic Regression (без torque/speed)",
    logreg_no_ts,
    X_no_ts,
    y,
    preprocess_no_ts,
    use_smote=False,
)

rf_no_ts = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=20,
    n_jobs=-1,
    class_weight="balanced_subsample",
    random_state=RANDOM_STATE,
)
rf_no_ts_scores = evaluate_model_cv(
    "RandomForest (без torque/speed)",
    rf_no_ts,
    X_no_ts,
    y,
    preprocess_no_ts,
    use_smote=False,
)


Сравнение метрик с предыдущими таблицами показывает, насколько полезно
хранить исходные `torque_nm` и `rot_speed_rpm` вместе с их производными
(power, torque_wear). Для древовидных моделей обычно вариант *с* исходными
признаками даёт лучшее качество, что подтверждает рекомендацию не удалять их
преждевременно.



## 10. Дополнительная задача: классификация **типа** отказа

Здесь реализуем предложенное усложнение:

- Вместо бинарного таргета `machine_failure` предсказываем **тип отказа**.
- Объединяем флаги `TWF/HDF/PWF/OSF/RNF` в один столбец `failure_type`.
- Работаем только с объектами, где:
- `machine_failure == 1`
- и **ровно один** из флагов равен 1 (иначе причина неоднозначна).

Это приводит к мультиклассовой классификации.
Из-за крайне редкого `RNF` мы объединяем все редкие классы в категорию "Other".


In [ ]:
failure_flags = ["TWF", "HDF", "PWF", "OSF", "RNF"]

# Оставляем только строки с отказами и ровно одной причиной
mask_one_fail = (
    (data_fe["machine_failure"] == 1)
    & (data_fe[failure_flags].sum(axis=1) == 1)
)
df_reason = data_fe[mask_one_fail].copy()
print("Число отказов с одной причиной:", df_reason.shape[0])

# Столбец типа отказа — индекс столбца с единицей
df_reason["failure_type"] = df_reason[failure_flags].idxmax(axis=1)

print("Распределение причин отказа:")
print(df_reason["failure_type"].value_counts())

# Объединим редкие причины (< 10 объектов) в класс "Other"
rare_types = df_reason["failure_type"].value_counts()[lambda s: s < 10].index
df_reason.loc[df_reason["failure_type"].isin(rare_types), "failure_type"] = "Other"

print("
После объединения редких классов:")
print(df_reason["failure_type"].value_counts())

# Признаки — как раньше, но без флагов и без machine_failure
mc_feature_cols = feature_cols_base  # берём те же признаки, что и для бинарной задачи
X_mc = df_reason[mc_feature_cols]
y_mc = df_reason["failure_type"]

X_train_mc, X_valid_mc, y_train_mc, y_valid_mc = train_test_split(
    X_mc,
    y_mc,
    test_size=0.2,
    stratify=y_mc,
    random_state=RANDOM_STATE,
)

preprocess_mc, num_mc, cat_mc = make_preprocessor(X_train_mc)

# Для мультиклассовой задачи используем логистическую регрессию и RandomForest
logreg_mc = LogisticRegression(
    max_iter=1000,
    multi_class="multinomial",
    class_weight="balanced",
)

rf_mc = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=10,
    n_jobs=-1,
    class_weight="balanced_subsample",
    random_state=RANDOM_STATE,
)

pipe_logreg_mc = Pipeline(
    steps=[
        ("preprocess", preprocess_mc),
        ("model", logreg_mc),
    ]
)

pipe_rf_mc = Pipeline(
    steps=[
        ("preprocess", preprocess_mc),
        ("model", rf_mc),
    ]
)

pipe_logreg_mc.fit(X_train_mc, y_train_mc)
pipe_rf_mc.fit(X_train_mc, y_train_mc)

y_pred_logreg_mc = pipe_logreg_mc.predict(X_valid_mc)
y_pred_rf_mc = pipe_rf_mc.predict(X_valid_mc)

print("
Мультиклассовая логистическая регрессия (тип отказа):")
print(classification_report(y_valid_mc, y_pred_logreg_mc))

print("
RandomForest (тип отказа):")
print(classification_report(y_valid_mc, y_pred_rf_mc))


Мультиклассовая постановка демонстрирует, что разные типы отказов действительно
зависят от разных факторов:
- HDF — от температурного режима;
- PWF — от нагрузки (torque, power);
- TWF — от износа инструмента;
- OSF — от комбинации факторов.

Это задание не обязательно по условию кейса, но хорошо иллюстрирует логику
работы с несбалансированными мультиклассовыми данными и показывает
взаимосвязь признаков и типов отказов.


# Итог

- Выполнен подробный EDA с нормированными гистограммами и анализом
корреляций.
- Обоснована и реализована инженерия признаков (разность температур, мощность,
взаимодействие нагрузки и износа).
- Для основной задачи предсказания `Machine failure` **флаги типов отказов**
(TWF, HDF, PWF, OSF, RNF) не используются как признаки, чтобы избежать
утечки таргета.
- Построен baseline (Logistic Regression) и проведено сравнение нескольких
моделей (Decision Tree, SVM, RandomForest, GradientBoosting, XGBoost,
LightGBM) на кросс-валидации с учётом дисбаланса классов.
- Проверено влияние удаления исходных признаков `torque_nm` и
`rot_speed_rpm` после добавления инженерных фич.
- В дополнительной части решена задача мультиклассовой классификации
типа отказа с объединением флагов в один столбец `failure_type`.